# KITTI Dataset augmentation visualization

## Dataset structure

```
📦gta
┗ 📂training
  ┣ 📂calib
  ┃ ┣ 📜<file_id>.txt
  ┃ ┗ 📜 ...
  ┣ 📂image_2
  ┃ ┣ 📜<file_id>.png
  ┃ ┗ 📜 ...
  ┣ 📂label_2
  ┃ ┣ 📜<file_id>.txt
  ┃ ┗ 📜 ...
  ┗ 📂velodyne
    ┣ 📜<file_id>.bin
    ┗ 📜 ...
```

In [1]:
import os
import mayavi.mlab as mlab
import numpy as np
from copy import deepcopy
import pickle
import logging

import pcdet.datasets.augmentor

from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
from pcdet.models import build_network
from tools.visual_utils import visualize_utils as V

mlab.init_notebook()

Notebook initialized with ipy backend.


In [2]:
# generate pickles from the kitti data
# os.system("python -m pcdet.datasets.kitti.kitti_dataset create_kitti_infos tools/cfgs/dataset_configs/kitti_dataset.yaml")

## Visualizing the LiDAR point cloud with labels

In [3]:
logger = common_utils.create_logger(log_level=logging.DEBUG)

In [5]:
cfg_from_yaml_file('tools/cfgs/dataset_configs/kitti_dataset.yaml', cfg)
# cfg_from_yaml_file('tools/cfgs/kitti_models/pointpillar_augs.yaml', cfg)

cfg.DATA_PATH = 'data/kitti'

train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg,
    class_names=['Car', 'Pedestrian', 'Cyclist'],
    batch_size=1,
    dist=False,
    workers=4,
    logger=logger,
    training=True,
    merge_all_iters_to_one_epoch=False,
    total_epochs=0
)

logger.info(f'Total number of samples: \t{len(train_set)}')

data_dict_list = []
logger.info('Loading samples')
for idx, data_dict in enumerate(train_set):
    logger.info(f'Loaded sample index: \t{idx + 1}')
    data_dict = train_set.collate_batch([data_dict])
    data_dict_list.append(data_dict)
    #if idx > 8: break


2021-05-14 23:47:08,889   INFO  Database filter by min points Car: 189 => 177
2021-05-14 23:47:08,890   INFO  Database filter by min points Pedestrian: 25 => 25
2021-05-14 23:47:08,891   INFO  Database filter by min points Cyclist: 10 => 10
2021-05-14 23:47:08,893   INFO  Database filter by difficulty Car: 177 => 137
2021-05-14 23:47:08,894   INFO  Database filter by difficulty Pedestrian: 25 => 24
2021-05-14 23:47:08,895   INFO  Database filter by difficulty Cyclist: 10 => 9
2021-05-14 23:47:08,972   INFO  Loading KITTI dataset
2021-05-14 23:47:08,983   INFO  Total samples for KITTI dataset: 58
2021-05-14 23:47:08,984   INFO  Total number of samples: 	87
2021-05-14 23:47:08,985   INFO  Loading samples


AssertionError: 

In [5]:
scene = data_dict_list[2]

In [6]:
def show_pc(data_dict):
    V.draw_scenes(
        points=data_dict['points'][:, 1:], gt_boxes=data_dict['gt_boxes'][0]
    )

    return mlab.test_plot3d()


## Visualizing the original scene
![Original Camera Image](data/kitti/training/image_2/000007.png)


In [7]:
show_pc(data_dict_list[60])
mlab.savefig('3d_renders/aug_pipeline_output.obj')
mlab.savefig('figs/aug_pipeline_output.png')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

## Augmentation methods

### Random flip

In [8]:
def random_flip_along_x(gt_boxes, points):
    """
    Args:
        gt_boxes: (N, 7 + C), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C)
    Returns:
    """
    enable = True
    if enable:
        gt_boxes[:, 1] = -gt_boxes[:, 1]
        gt_boxes[:, 6] = -gt_boxes[:, 6]
        points[:, 1] = -points[:, 1]

        if gt_boxes.shape[1] > 8:
            gt_boxes[:, 8] = -gt_boxes[:, 8]

    return gt_boxes, points

def random_flip_along_y(gt_boxes, points):
    """
    Args:
        gt_boxes: (N, 7 + C), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C)
    Returns:
    """
    enable = True
    if enable:
        gt_boxes[:, 0] = -gt_boxes[:, 0]
        gt_boxes[:, 6] = -(gt_boxes[:, 6] + np.pi)
        points[:, 0] = -points[:, 0]

        if gt_boxes.shape[1] > 7:
            gt_boxes[:, 7] = -gt_boxes[:, 7]

    return gt_boxes, points

In [9]:
%%time
flipped_y = deepcopy(scene)
flipped_y['gt_boxes'][0], flipped_y['points'][:,1:] = random_flip_along_y(flipped_y['gt_boxes'][0], flipped_y['points'][:,1:])

CPU times: user 1.51 ms, sys: 751 µs, total: 2.26 ms
Wall time: 1.26 ms


In [10]:
show_pc(flipped_y)
mlab.savefig('figs/flipped_y.png')
mlab.savefig('3d_renders/flipped_y.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [11]:
%%time
flipped_x = deepcopy(scene)
flipped_x['gt_boxes'][0], flipped_x['points'][:,1:] = random_flip_along_x(flipped_x['gt_boxes'][0], flipped_x['points'][:,1:])

CPU times: user 1.65 ms, sys: 750 µs, total: 2.4 ms
Wall time: 1.29 ms


In [12]:
show_pc(flipped_x)
mlab.savefig('figs/flipped_x.png')
mlab.savefig('3d_renders/flipped_x.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Global rotation

In [13]:
def global_rotation(gt_boxes, points, rot_range):
    """
    Args:
        gt_boxes: (N, 7 + C), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        rot_range: [min, max]
    Returns:
    """
    noise_rotation = np.random.uniform(rot_range[0], rot_range[1])
    points = common_utils.rotate_points_along_z(points[np.newaxis, :, :], np.array([noise_rotation]))[0]
    gt_boxes[:, 0:3] = common_utils.rotate_points_along_z(gt_boxes[np.newaxis, :, 0:3], np.array([noise_rotation]))[0]
    gt_boxes[:, 6] += noise_rotation
    if gt_boxes.shape[1] > 8:
        gt_boxes[:, 7:9] = common_utils.rotate_points_along_z(
            np.hstack((gt_boxes[:, 7:9], np.zeros((gt_boxes.shape[0], 1))))[np.newaxis, :, :],
            np.array([noise_rotation])
        )[0][:, 0:2]

    return gt_boxes, points

In [14]:
%%time
rotated = deepcopy(scene)
rotated['gt_boxes'][0], rotated['points'][:,1:] = global_rotation(rotated['gt_boxes'][0], rotated['points'][:,1:], [-0.78539816, 0.78539816])

CPU times: user 4.52 ms, sys: 870 µs, total: 5.39 ms
Wall time: 3.02 ms


In [15]:
show_pc(rotated)
mlab.savefig('figs/rotated_global.png')
mlab.savefig('3d_renders/rotated_global.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Global scaling

In [16]:
def global_scaling(gt_boxes, points, scale_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading]
        points: (M, 3 + C),
        scale_range: [min, max]
    Returns:
    """
    if scale_range[1] - scale_range[0] < 1e-3:
        return gt_boxes, points
    noise_scale = np.random.uniform(scale_range[0], scale_range[1])
    points[:, :3] *= noise_scale
    gt_boxes[:, :6] *= noise_scale
    return gt_boxes, points

In [17]:
%%time
scaled = deepcopy(scene)
scaled['gt_boxes'][0], scaled['points'][:,1:] = global_scaling(scaled['gt_boxes'][0], scaled['points'][:, 1:], [0.2, 0.5])

CPU times: user 3.04 ms, sys: 0 ns, total: 3.04 ms
Wall time: 1.86 ms


In [18]:
show_pc(scaled)
mlab.savefig('figs/scaled_global.png')
mlab.savefig('3d_renders/scaled_global.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Global translation

In [19]:
def random_translation_along_x(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    offset = np.random.uniform(offset_range[0], offset_range[1])

    points[:, 0] += offset
    gt_boxes[:, 0] += offset

    if gt_boxes.shape[1] > 7:
        gt_boxes[:, 7] += offset

    return gt_boxes, points

def random_translation_along_y(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    offset = np.random.uniform(offset_range[0], offset_range[1])

    points[:, 1] += offset
    gt_boxes[:, 1] += offset

    if gt_boxes.shape[1] > 8:
        gt_boxes[:, 8] += offset

    return gt_boxes, points

def random_translation_along_z(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    offset = np.random.uniform(offset_range[0], offset_range[1])

    points[:, 2] += offset
    gt_boxes[:, 2] += offset

    return gt_boxes, points

In [20]:
%%time
translation_x = deepcopy(scene)
translation_x['gt_boxes'][0], translation_x['points'][:,1:] = random_translation_along_x(translation_x['gt_boxes'][0], translation_x['points'][:, 1:], [15, 20])

CPU times: user 1.78 ms, sys: 648 µs, total: 2.43 ms
Wall time: 1.26 ms


In [21]:
show_pc(translation_x)
mlab.savefig('figs/translation_global_x.png')
mlab.savefig('3d_renders/translation_global_x.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [22]:
%%time
translation_y = deepcopy(scene)
translation_y['gt_boxes'][0], translation_y['points'][:,1:] = random_translation_along_y(translation_y['gt_boxes'][0], translation_y['points'][:, 1:], [15, 20])

CPU times: user 2.1 ms, sys: 0 ns, total: 2.1 ms
Wall time: 1.12 ms


In [23]:
show_pc(translation_y)
mlab.savefig('figs/translation_global_y.png')
mlab.savefig('3d_renders/translation_global_y.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [24]:
%%time
translation_z = deepcopy(scene)
translation_z['gt_boxes'][0], translation_z['points'][:,1:] = random_translation_along_z(translation_z['gt_boxes'][0], translation_z['points'][:, 1:], [15, 20])

CPU times: user 1.87 ms, sys: 607 µs, total: 2.48 ms
Wall time: 1.42 ms


In [25]:
show_pc(translation_z)
mlab.savefig('figs/translation_global_z.png')
mlab.savefig('3d_renders/translation_global_z.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Global frustum dropout

In [26]:
def global_frustum_dropout_top(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    intensity = np.random.uniform(intensity_range[0], intensity_range[1])

    threshold = np.max(points[:, 2]) - intensity * (np.max(points[:, 2]) - np.min(points[:, 2]))
    points = points[points[:,2] < threshold]
    gt_boxes = gt_boxes[gt_boxes[:,2] < threshold]

    return gt_boxes, points

def global_frustum_dropout_bottom(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    intensity = np.random.uniform(intensity_range[0], intensity_range[1])

    threshold = np.min(points[:, 2]) + intensity * (np.max(points[:, 2]) - np.min(points[:, 2]))
    points = points[points[:,2] > threshold]
    gt_boxes = gt_boxes[gt_boxes[:,2] > threshold]

    return gt_boxes, points

def global_frustum_dropout_left(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    intensity = np.random.uniform(intensity_range[0], intensity_range[1])

    threshold = np.max(points[:, 1]) - intensity * (np.max(points[:, 1]) - np.min(points[:, 1]))
    points = points[points[:,1] < threshold]
    gt_boxes = gt_boxes[gt_boxes[:,1] < threshold]

    return gt_boxes, points

def global_frustum_dropout_right(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    intensity = np.random.uniform(intensity_range[0], intensity_range[1])

    threshold = np.min(points[:, 1]) + intensity * (np.max(points[:, 1]) - np.min(points[:, 1]))
    points = points[points[:,1] > threshold]
    gt_boxes = gt_boxes[gt_boxes[:,1] > threshold]

    return gt_boxes, points

In [27]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = global_frustum_dropout_top(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.4, 0.5])

CPU times: user 3.35 ms, sys: 0 ns, total: 3.35 ms
Wall time: 2.19 ms


In [28]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/frustum_dropout_top.png')
mlab.savefig('3d_renders/frustum_dropout_top.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [29]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = global_frustum_dropout_bottom(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.4, 0.5])

CPU times: user 2.61 ms, sys: 759 µs, total: 3.36 ms
Wall time: 2.29 ms


In [30]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/frustum_dropout_bottom.png')
mlab.savefig('3d_renders/frustum_dropout_bottom.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [31]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = global_frustum_dropout_left(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.4, 0.5])

CPU times: user 2.44 ms, sys: 692 µs, total: 3.13 ms
Wall time: 1.86 ms


In [32]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/frustum_dropout_left.png')
mlab.savefig('figs/frustum_dropout_bottom.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [33]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = global_frustum_dropout_right(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.4, 0.5])

CPU times: user 3.34 ms, sys: 0 ns, total: 3.34 ms
Wall time: 2.21 ms


In [34]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/frustum_dropout_right.png')
mlab.savefig('figs/frustum_dropout_right.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

#### Aux function to find which points are inside a bounding box

In [35]:
def get_points_in_box(points, gt_box):
    x, y, z, dx, dy, dz = gt_box[0], gt_box[1], gt_box[2], gt_box[3], gt_box[4], gt_box[5]

    mask = np.logical_and(points[:,0] <= x + dx/2, \
         np.logical_and(points[:,0] >= x - dx/2, \
             np.logical_and(points[:,1] <= y + dy/2, \
                 np.logical_and(points[:,1] >= y - dy/2, \
                     np.logical_and(points[:,2] <= z + dz/2, points[:,2] >= z - dz/2)))))

    points = points[mask]

    return points, mask

In [36]:
%%time
points_to_filter = deepcopy(scene)
filtered_points, mask = get_points_in_box(points_to_filter['points'][:,1:], points_to_filter['gt_boxes'][0][1])

CPU times: user 2.94 ms, sys: 0 ns, total: 2.94 ms
Wall time: 1.95 ms


In [37]:
V.draw_scenes(
        points=filtered_points, gt_boxes=points_to_filter['gt_boxes'][0,1:]
    )

mlab.savefig('figs/points_in_box.png')
mlab.savefig('3d_renders/points_in_box.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Local translation

In [38]:
def local_translation_along_x(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        offset = np.random.uniform(offset_range[0], offset_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        points[mask, 0] += offset

        gt_boxes[idx, 0] += offset

        if gt_boxes.shape[1] > 7:
            gt_boxes[idx, 7] += offset

    return gt_boxes, points


def local_translation_along_y(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        offset = np.random.uniform(offset_range[0], offset_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        points[mask, 1] += offset

        gt_boxes[idx, 1] += offset

        if gt_boxes.shape[1] > 8:
            gt_boxes[idx, 8] += offset

    return gt_boxes, points


def local_translation_along_z(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        offset = np.random.uniform(offset_range[0], offset_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        points[mask, 2] += offset

        gt_boxes[idx, 2] += offset

    return gt_boxes, points

In [39]:
%%time
local_translation_x = deepcopy(scene)
local_translation_x['gt_boxes'][0], local_translation_x['points'][:,1:] = local_translation_along_x(local_translation_x['gt_boxes'][0], local_translation_x['points'][:, 1:], [10, 15])

CPU times: user 6.22 ms, sys: 0 ns, total: 6.22 ms
Wall time: 4.99 ms


In [40]:
show_pc(local_translation_x)
mlab.savefig('figs/translation_local_x.png')
mlab.savefig('3d_renders/translation_local_x.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [41]:
%%time
local_translation_y = deepcopy(scene)
local_translation_y['gt_boxes'][0], local_translation_y['points'][:,1:] = local_translation_along_y(local_translation_y['gt_boxes'][0], local_translation_y['points'][:, 1:], [10, 15])

CPU times: user 2.06 ms, sys: 2.56 ms, total: 4.62 ms
Wall time: 3.44 ms


In [42]:
show_pc(local_translation_y)
mlab.savefig('figs/translation_local_y.png')
mlab.savefig('3d_renders/translation_local_y.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [43]:
%%time
local_translation_z = deepcopy(scene)
local_translation_z['gt_boxes'][0], local_translation_z['points'][:,1:] = local_translation_along_z(local_translation_z['gt_boxes'][0], local_translation_z['points'][:, 1:], [10, 15])

CPU times: user 5.21 ms, sys: 0 ns, total: 5.21 ms
Wall time: 4.04 ms


In [44]:
show_pc(local_translation_z)
mlab.savefig('figs/translation_local_z.png')
mlab.savefig('3d_renders/translation_local_z.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Local scaling

In [45]:
def local_scaling(gt_boxes, points, scale_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading]
        points: (M, 3 + C),
        scale_range: [min, max]
    Returns:
    """
    if scale_range[1] - scale_range[0] < 1e-3:
        return gt_boxes, points

    for idx, box in enumerate(gt_boxes):
        noise_scale = np.random.uniform(scale_range[0], scale_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        
        # tranlation to axis center
        points[mask, 0] -= box[0]
        points[mask, 1] -= box[1]
        points[mask, 2] -= box[2]

        # apply scaling
        points[mask, :3] *= noise_scale

        # tranlation back to original position
        points[mask, 0] += box[0]
        points[mask, 1] += box[1]
        points[mask, 2] += box[2]

        gt_boxes[idx, 3:6] *= noise_scale
    return gt_boxes, points

In [46]:
%%time
scaled_local = deepcopy(scene)
scaled_local['gt_boxes'][0], scaled_local['points'][:,1:] = local_scaling(scaled_local['gt_boxes'][0], scaled_local['points'][:, 1:], [0.2, 0.5])

CPU times: user 8.89 ms, sys: 196 µs, total: 9.09 ms
Wall time: 7.88 ms


In [47]:
show_pc(scaled_local)
mlab.savefig('figs/scaled_local.png')
mlab.savefig('3d_renders/scaled_local.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Local rotation

In [48]:
def local_rotation(gt_boxes, points, rot_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        rot_range: [min, max]
    Returns:
    """
    
    for idx, box in enumerate(gt_boxes):
        noise_rotation = np.random.uniform(rot_range[0], rot_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        
        centroid_x = box[0]
        centroid_y = box[1]
        centroid_z = box[2]

        # tranlation to axis center
        points[mask, 0] -= centroid_x
        points[mask, 1] -= centroid_y
        points[mask, 2] -= centroid_z
        box[0] -= centroid_x
        box[1] -= centroid_y
        box[2] -= centroid_z

        # apply rotation
        points[mask, :] = common_utils.rotate_points_along_z(points[np.newaxis, mask, :], np.array([noise_rotation]))[0]
        box[0:3] = common_utils.rotate_points_along_z(box[np.newaxis, np.newaxis, 0:3], np.array([noise_rotation]))[0][0]

        # tranlation back to original position
        points[mask, 0] += centroid_x
        points[mask, 1] += centroid_y
        points[mask, 2] += centroid_z
        box[0] += centroid_x
        box[1] += centroid_y
        box[2] += centroid_z

        gt_boxes[idx, 6] += noise_rotation
        if gt_boxes.shape[1] > 8:
            gt_boxes[idx, 7:9] = common_utils.rotate_points_along_z(
                np.hstack((gt_boxes[idx, 7:9], np.zeros((gt_boxes.shape[0], 1))))[np.newaxis, :, :],
                np.array([noise_rotation])
            )[0][:, 0:2]

    return gt_boxes, points

In [49]:
%%time
rotated_local = deepcopy(scene)
rotated_local['gt_boxes'][0], rotated_local['points'][:,1:] = local_rotation(rotated_local['gt_boxes'][0],rotated_local['points'][:, 1:], [-0.78539815, 0.78539816])

CPU times: user 10.4 ms, sys: 513 µs, total: 10.9 ms
Wall time: 9.42 ms


In [50]:
show_pc(rotated_local)
mlab.savefig('figs/rotated_local.png')
mlab.savefig('3d_renders/rotated_local.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

### Local frustum dropout

In [51]:
def local_frustum_dropout_top(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        x, y, z, dx, dy, dz = box[0], box[1], box[2], box[3], box[4], box[5]

        intensity = np.random.uniform(intensity_range[0], intensity_range[1])
        points_in_box = get_points_in_box(points, box)
        threshold = (z + dz/2) - intensity * dz

        points = points[np.logical_not( \
        np.logical_and(points[:,1] <= y + dy/2, \
        np.logical_and(points[:,1] >= y - dy/2, \
            np.logical_and(points[:,0] <= x + dx/2, \
                np.logical_and(points[:,0] >= x - dx/2, \
                    np.logical_and(points[:,2] <= z + dz/2, points[:,2] >= threshold))))))]

    return gt_boxes, points

def local_frustum_dropout_bottom(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        x, y, z, dx, dy, dz = box[0], box[1], box[2], box[3], box[4], box[5]

        intensity = np.random.uniform(intensity_range[0], intensity_range[1])
        points_in_box = get_points_in_box(points, box)
        threshold = (z - dz/2) + intensity * dz
        points = points[np.logical_not( \
            np.logical_and(points[:,1] <= y + dy/2, \
            np.logical_and(points[:,1] >= y - dy/2, \
                np.logical_and(points[:,0] <= x + dx/2, \
                    np.logical_and(points[:,0] >= x - dx/2, \
                        np.logical_and(points[:,2] <= threshold, points[:,2] >= z - dz/2))))))]

    return gt_boxes, points

def local_frustum_dropout_left(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        x, y, z, dx, dy, dz = box[0], box[1], box[2], box[3], box[4], box[5]

        intensity = np.random.uniform(intensity_range[0], intensity_range[1])
        points_in_box = get_points_in_box(points, box)
        threshold = (y + dy/2) - intensity * dy

        points = points[np.logical_not( \
            np.logical_and(points[:,1] <= y + dy/2, \
            np.logical_and(points[:,1] >= threshold, \
                np.logical_and(points[:,0] <= x + dx/2, \
                    np.logical_and(points[:,0] >= x - dx/2, \
                        np.logical_and(points[:,2] <= z + dz/2, points[:,2] >= z - dz/2))))))]

    return gt_boxes, points

def local_frustum_dropout_right(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        x, y, z, dx, dy, dz = box[0], box[1], box[2], box[3], box[4], box[5]

        intensity = np.random.uniform(intensity_range[0], intensity_range[1])
        points_in_box = get_points_in_box(points, box)
        threshold = (y - dy/2) + intensity * dy

        points = points[np.logical_not( \
            np.logical_and(points[:,1] <= threshold, \
            np.logical_and(points[:,1] >= y - dy/2, \
                np.logical_and(points[:,0] <= x + dx/2, \
                    np.logical_and(points[:,0] >= x - dx/2, \
                        np.logical_and(points[:,2] <= z + dz/2, points[:,2] >= z - dz/2))))))]

    return gt_boxes, points

In [52]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = local_frustum_dropout_top(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.5, 0.5])

CPU times: user 7.38 ms, sys: 699 µs, total: 8.08 ms
Wall time: 6.87 ms


In [53]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/local_frustum_dropout_top.png')
mlab.savefig('3d_renders/local_frustum_dropout_top.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [54]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = local_frustum_dropout_bottom(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.3, 0.4])

CPU times: user 4.54 ms, sys: 2.8 ms, total: 7.34 ms
Wall time: 6.2 ms


In [55]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/local_frustum_dropout_bottom.png')
mlab.savefig('3d_renders/local_frustum_dropout_bottom.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [56]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = local_frustum_dropout_left(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.3, 0.4])

CPU times: user 9.18 ms, sys: 224 µs, total: 9.4 ms
Wall time: 8.09 ms


In [57]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/local_frustum_dropout_left.png')
mlab.savefig('3d_renders/local_frustum_dropout_left.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…

In [58]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = local_frustum_dropout_right(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.3, 0.4])

CPU times: user 3.53 ms, sys: 7.18 ms, total: 10.7 ms
Wall time: 9.18 ms


In [59]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/local_frustum_dropout_right.png')
mlab.savefig('3d_renders/local_frustum_dropout_right.obj')
mlab.test_plot3d()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x02\x00\x00\x001\x04\x0f\x8b\x…